In [2]:
import json, wandb
from importlib import reload

In [1]:
config={"data_params":{ 
    "n_data": 1e7,
    "graph_construction":       "classic",
    "muon":             False,
    "n_neighbors":       10,
    "restart":   False,
    "transform_path": "../../../../pcs557/databases/dev_lvl7_mu_nu_e_classification_v003/meta/transformers.pkl",
    "db_path": "../../../../pcs557/databases/dev_lvl7_mu_nu_e_classification_v003/data/dev_lvl7_mu_nu_e_classification_v003.db",
    "features":   ["dom_x", "dom_y", "dom_z", "dom_time", "charge_log10", "width", "rqe"],
    "targets":    ["energy_log10", "zenith","azimuth"],
    "database": "submit",
    "steps": 2,
    "n_test": 100000
}}

In [31]:
import dev.submit_testdata as dl
reload(dl)
data=dl.graph_data(**config['data_params'])

100%|██████████| 50000/50000 [00:26<00:00, 1856.13it/s]
Saving dataset 0
 50%|█████     | 1/2 [07:47<07:47, 467.01s/it]Reading 50000 events
Connecting to db-file
Reading files
   event_no
0   7131083
1   7131083
2   7131083
3   7131083
4   7131083
Events read
Features read
Truth read
Splitting data to events

  0%|          | 0/50000 [00:00<?, ?it/s]     dom_x    dom_y    dom_z  dom_time  charge_log10  width   rqe
0  0.03125 -0.07293 -0.35537  0.455056     -0.250000    1.0  0.00
1  0.03125 -0.07293 -0.41845  0.500000     -0.833333    1.0 -0.35
2  0.03125 -0.07293 -0.43947 -0.018727      0.500000    1.0 -0.35
3  0.07237 -0.06660 -0.37710 -0.140449     -0.250000    1.0  0.00
4  0.07237 -0.06660 -0.41915 -0.314607     -1.166667    0.0 -0.35
   energy_log10    zenith   azimuth  event_no
0      0.528218  2.644635  3.804066   7131083
1      0.588507  1.428631  5.964864   7131107
2      0.868192  2.913747  4.234655   7131142
3      0.867350  1.692980  0.332238   7131205
4      0.626248  1.953

In [34]:
data[0].y

array([5.76440513e-01, 1.17941677e+00, 1.51179985e+00, 7.13101600e+06])

In [22]:
from spektral.data import DisjointLoader
loader_train = DisjointLoader(data, epochs=1, batch_size=1e4)

In [23]:
loader_train.tf_signature()

((TensorSpec(shape=(None, 7), dtype=tf.float64, name=None),
  SparseTensorSpec(TensorShape([None, None]), tf.float64),
  TensorSpec(shape=(None,), dtype=tf.int64, name=None)),
 TensorSpec(shape=(None, 3), dtype=tf.float64, name=None))

In [3]:
def get_event_no(set_path='../../../../pcs557/databases/dev_lvl7_mu_nu_e_classification_v003/meta/sets.pkl'):
    import pandas as pd
    print('Reading sets')
    sets = pd.read_pickle(set_path)
    train_events = sets['train']
    test_events = sets['test']
    return train_events['event_no'].to_numpy(), test_events['event_no'].to_numpy()

train_events, test_events = get_event_no()
idx_lists = [np.array(df[df['event_no'].isin(train_events)].index), np.array(df[df['event_no'].isin(test_events)].index)] 
print(len(idx_lists[0]), max(idx_lists[0]), len(idx_lists[1]), max(idx_lists[1]))
# Split data
# dataset_train = dataset[idx_lists[0]]
print(len(dataset))
dataset_train = dataset[idx_lists[0][:-4]]
dataset_test = dataset[idx_lists[1]]

Reading sets


NameError: name 'np' is not defined

In [ ]:
gpu_devices = tf.config.list_physical_devices('GPU') 
if len(gpu_devices) > 0:
    print("GPU detected")
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)

In [4]:
configtr={"data_params":{ 
    "n_data": 1e7,
    "graph_construction":       "classic",
    "muon":             False,
    "n_neighbors":       10,
    "restart":   True,
    "transform_path": "../../../../pcs557/databases/dev_lvl7_mu_nu_e_classification_v003/meta/transformers.pkl",
    "db_path": "../../../../pcs557/databases/dev_lvl7_mu_nu_e_classification_v003/data/dev_lvl7_mu_nu_e_classification_v003.db",
    "features":   ["dom_x", "dom_y", "dom_z", "dom_time", "charge_log10", "width", "rqe"],
    "targets":    ["energy_log10", "zenith","azimuth"],
    "database": "submit",
    "train_steps": 2,
    "n_train": 10000
}}

In [5]:
import numpy as np
import os, sqlite3, pickle, sys, gzip, shutil
if hasattr(__builtins__,'__IPYTHON__'):
    print('Notebook')
    from tqdm.notebook import tqdm
else:
    print('Not notebook')
    from tqdm import tqdm
import os.path as osp

from pandas import read_sql, concat
import pandas as pd
from sklearn.preprocessing import normalize, RobustScaler
from sklearn.neighbors import kneighbors_graph as knn
import matplotlib.pyplot as plt

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
from spektral.data import Dataset, Graph
from scipy.sparse import csr_matrix

class train_data():
    """
    data that takes config file
    """

    def __init__(self,features=["dom_x", "dom_y", "dom_z", "dom_time", "charge_log10", "width", "rqe"], 
    targets= ["energy_log10", "zenith","azimuth"],
    transform_path='../db_files/dev_lvl7/transformers.pkl',
    db_path= '../db_files/dev_lvl7/dev_lvl7_mu_nu_e_classification_v003.db', 
    set_path="../db_files/dev_lvl7/sets.pkl",
    n_neighbors = 30, restart=False, n_train='all', graph_construction='classic', database='submit', train_steps=20, **kwargs):


        self.features=features
        self.targets=targets
        self.dom_norm = 1e3
        self.transform_path=transform_path
        self.db_path=db_path
        self.set_path=set_path
        self.n_neighbors = n_neighbors
        self.seed = 42
        self.n_train=n_train
        self.restart=restart
        self.graph_construction=graph_construction
        self.database=database
        self.steps=train_steps
    
    @property
    def path(self):
        """
        Set the path of the data to be in the processed folder
        """
        cwd = osp.abspath('')
        path = osp.join(cwd, f"processed/{self.database}_{self.n_neighbors}nn_{self.graph_construction}graph_{len(self.features)}feat_train")
        return path

    def reload(self):
        if os.path.isdir(self.path):
            shutil.rmtree(self.path)
            print('Removed and ready to reload')

    def get_event_no(self):
        print('Reading sets')
        sets = pd.read_pickle(self.set_path)
        train_events = sets['train']
        test_events = sets['test']
        return train_events['event_no'].to_numpy(), test_events['event_no'].to_numpy()

    def download(self):
        # Get raw_data
        # Make output folder
        # os.makedirs(self.path)
        train_events, _=self.get_event_no()
        if self.n_train=='all':
            events=train_events
        else:
            # events=train_events[:self.n_train]
            events=train_events[:self.n_train]
        events=np.array_split(events, self.steps)
        for i in tqdm(range(self.steps)):
            eventsi=events[i]
            print(f'Reading {len(eventsi)} events')
            db_file  = self.db_path
            print("Connecting to db-file")
            with sqlite3.connect(db_file) as conn:
                # SQL queries format
                feature_call = ", ".join(self.features)
                target_call  = ", ".join(self.targets)
                event_nos=tuple(eventsi.reshape(1, -1)[0])
                # Load data from db-file
                print("Reading files")
                df_event = read_sql(f"select event_no from features where event_no in {event_nos}", conn)
                self.df_event=df_event
                print(self.df_event.head())
                print("Events read")
                df_feat  = read_sql(f"select {feature_call} from features where event_no in {event_nos}", conn)
                self.df_feat=df_event
                print("Features read")
                df_targ  = read_sql(f"select {target_call} from truth where event_no in {event_nos}", conn)
                self.df_targ=df_targ
                print("Truth read")

                transformers = pickle.load(open(self.transform_path, 'rb'))
                trans_x      = transformers['features']
                trans_y      = transformers['truth']


                for col in ["dom_x", "dom_y", "dom_z"]:
                    df_feat[col] = trans_x[col].inverse_transform(np.array(df_feat[col]).reshape(1, -1)).T/self.dom_norm

                for col in df_targ.columns:
                    df_targ[col] = trans_y[col].inverse_transform(np.array(df_targ[col]).reshape(1, -1)).T
            
            

                # Cut indices
                print("Splitting data to events")
                idx_list    = np.array(df_event)
                x_not_split = np.array(df_feat)

                _, idx, counts = np.unique(idx_list.flatten(), return_index = True, return_counts = True) 
                self.counts=counts
                self.idx=idc´x
                xs          = np.split(x_not_split, np.cumsum(counts)[:-1])
                self.xs=xs

                ys          = np.array(df_targ)
                self.ys=ys
                print(df_feat.head())
                print(df_targ.head())

                # Generate adjacency matrices
                print("Generating adjacency matrices")
                graph_list = []
                for x, y in tqdm(zip(xs, ys), total = len(xs)):
                    try:
                        a = knn(x[:, :3], self.n_neighbors)
                    except:
                        a = csr_matrix(np.ones(shape = (x.shape[0], x.shape[0])) - np.eye(x.shape[0]))


                    graph_list.append(Graph(x = x, a = a, y = y))

                graph_list = np.array(graph_list, dtype = object)

Notebook


In [6]:
dat=train_data(**configtr['data_params'])

In [7]:
dat.download()

Reading sets


Reading 5000 events
Connecting to db-file
Reading files
   event_no
0  13188817
1  13188817
2  13188817
3  13188817
4  13188817
Events read
Features read
Truth read
Splitting data to events
     dom_x    dom_y    dom_z  dom_time  charge_log10  width   rqe
0  0.00171 -0.15063 -0.28570 -0.659176     -0.666667    1.0 -0.35
1  0.03125 -0.07293 -0.17315  0.262172      0.583333    1.0 -0.35
2  0.03125 -0.07293 -0.25725  0.413858     -0.333333    0.0 -0.35
3  0.03125 -0.07293 -0.27127  0.116105     -0.166667    0.0 -0.35
4  0.03125 -0.07293 -0.31332 -0.043071      0.166667    1.0  0.00
   energy_log10    zenith   azimuth
0      2.340637  1.027825  4.888147
1      2.416496  1.107861  4.361933
2      2.409569  1.011339  4.044844
3      2.333035  1.197129  1.402828
4      2.490171  0.406509  3.207328
Generating adjacency matrices



Reading 5000 events
Connecting to db-file
Reading files
   event_no
0  13188856
1  13188856
2  13188856
3  13188856
4  13188856
Events read
Features read
Truth read
Splitting data to events
     dom_x    dom_y    dom_z  dom_time  charge_log10  width   rqe
0  0.12497 -0.13125 -0.18718 -0.108614     -0.083333    1.0 -0.35
1  0.12497 -0.13125 -0.25526 -0.132959      0.500000    1.0 -0.35
2  0.04629 -0.03488 -0.29900 -0.741573      0.166667    1.0  0.00
3  0.04629 -0.03488 -0.31602  1.694757      0.333333    1.0  0.00
4  0.04629 -0.03488 -0.36708  0.355805      0.000000    1.0  0.00
   energy_log10    zenith   azimuth
0      2.405540  0.917276  5.294104
1      2.360640  0.970332  4.417976
2      2.261745  1.057187  5.064054
3      2.319877  1.073880  1.410094
4      2.400316  0.927781  2.317087
Generating adjacency matrices


In [18]:
dat.df_event[40:60]

,event_no
40,13188856
41,13188856
42,13188856
43,13188856
44,13188856
45,13188856
46,13188856
47,13188856
48,13188856
49,13188905


In [9]:
dat.df_feat

,event_no
0,13188856
1,13188856
2,13188856
3,13188856
4,13188856
...,...
105093,117104763
105094,117104763
105095,117104763
105096,117104763
